## Get data

1. Download SI-NLI from [link](https://www.clarin.si/repository/xmlui/handle/11356/1707).
2. Load libraries.
3. Import ```train.tsv``` file.

In [1]:
import pandas as pd

df = pd.read_csv('SI-NLI/train.tsv', sep='\t')
df.head()

,pair_id,premise,hypothesis,annotation_1,comment_1,annotator1_id,annotation_2,comment_2,annotator2_id,annotation_3,comment_3,annotator3_id,annotation_FINAL,label
0,P0,Vendar se je anglikanska večina v grofijah na ...,A na glasovanju o priključitvi ozemlja k Sever...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment
1,P1,INŠTRUKTOR IZ PRTLJAŽNIKA V DRUGO POTOVALKO PR...,Učitelj je vzel iz prtljažnika iz prve potoval...,C,NaN,annotator_B,N,NaN,annotator_A,NaN,NaN,NaN,NaN,contradiction
2,P2,biotska raznovrstnost – v splošnem je to razno...,Četudi je biodiverziteta pomemben del biološke...,C,NaN,annotator_G,N,Morda neustrezen primer? (zdi se kot alineja),annotator_A,NaN,NaN,NaN,NaN,contradiction
3,P3,"Preroški pomen: Če v sanjah bedite, je to na s...",V preroškem smislu budnost v sanjah nakazuje o...,E,Pomanjkanje konteksta?,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment
4,P4,"Jeseni so dnevi krajši, stemni se že dokaj zgo...",V krajših jesenskih dneh tema nastopi relativn...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment


## Basic statistics

1. How many examples are in a dataframe?
2. How many variables are in a dataframe?
3. Count values in the ```label``` column.
4. Are there any missing values in the data?
5. Count the number of missing values per column.

In [2]:
# 1. How many examples are in a dataframe?
len(df)

4392

In [3]:
# 2. How many variables are in a dataframe?
len(df.columns)

14

In [4]:
# 3. Count values in the ```label``` column.
df['label'].value_counts()

label
entailment       1518
contradiction    1448
neutral          1426
Name: count, dtype: int64

In [5]:
# 4. Are there any missing values in the data?
any(df.isna())

True

In [6]:
# 5. Count the number of missing values per column.
df.isna().sum()

pair_id                0
premise                0
hypothesis             0
annotation_1          91
comment_1           4349
annotator1_id         91
annotation_2          91
comment_2           4274
annotator2_id         91
annotation_3        3981
comment_3           4384
annotator3_id       3976
annotation_FINAL     508
label                  0
dtype: int64

## Subset observations and variables

1. Select ```premise``` column and store it in a list.
2. Print first 3 rows from the first 3 columns.
3. Select ```pair_id```, ```premise```, ```hypothesis```, ```label``` columns and save them into ```train_dataset``` variable.
4. Drop ```pair_id``` column.
5. Convert all columns to uppercase.
6. Replace ```_``` with ```-``` in column names.
7. Select rows that belong to the ```neutral``` label.
8. Select last 30 rows.
9. Select rows with ```hypothesis``` longer than 100 characters.
10. Select rows with ```hypothesis``` longer than 100 characters and belong to the ```neutral``` label.
11. Select the row with the longest ```hypothesis```.
12. Remove rows that contain ```č```, ```š```, ```ž``` in ```premise``` or ```hypothesis```.
13. Remove rows that contain at least one missing value.
14. Remove the column with the most missing values.

In [7]:
# 1. Select premise column and store it in a list
premise_col = df['premise'].to_list()

In [8]:
# 2. Print first 3 rows from the first 3 columns.
df.iloc[:3, [0, 1, 2]]

,pair_id,premise,hypothesis
0,P0,Vendar se je anglikanska večina v grofijah na ...,A na glasovanju o priključitvi ozemlja k Sever...
1,P1,INŠTRUKTOR IZ PRTLJAŽNIKA V DRUGO POTOVALKO PR...,Učitelj je vzel iz prtljažnika iz prve potoval...
2,P2,biotska raznovrstnost – v splošnem je to razno...,Četudi je biodiverziteta pomemben del biološke...


In [9]:
# 3. Select ```pair_id```, ```premise```, ```hypothesis```, ```label``` columns and save them into ```train_dataset``` variable.
train_dataset = df[['pair_id', 'premise', 'hypothesis', 'label']]

In [10]:
# 4. Drop ```pair_id``` column.
df.drop(columns=['pair_id'])

,premise,hypothesis,annotation_1,comment_1,annotator1_id,annotation_2,comment_2,annotator2_id,annotation_3,comment_3,annotator3_id,annotation_FINAL,label
0,Vendar se je anglikanska večina v grofijah na ...,A na glasovanju o priključitvi ozemlja k Sever...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment
1,INŠTRUKTOR IZ PRTLJAŽNIKA V DRUGO POTOVALKO PR...,Učitelj je vzel iz prtljažnika iz prve potoval...,C,NaN,annotator_B,N,NaN,annotator_A,NaN,NaN,NaN,NaN,contradiction
2,biotska raznovrstnost – v splošnem je to razno...,Četudi je biodiverziteta pomemben del biološke...,C,NaN,annotator_G,N,Morda neustrezen primer? (zdi se kot alineja),annotator_A,NaN,NaN,NaN,NaN,contradiction
3,"Preroški pomen: Če v sanjah bedite, je to na s...",V preroškem smislu budnost v sanjah nakazuje o...,E,Pomanjkanje konteksta?,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment
4,"Jeseni so dnevi krajši, stemni se že dokaj zgo...",V krajših jesenskih dneh tema nastopi relativn...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4387,"Kot je povedal, naj bi novi načrt podpisal tak...",Sam je brez zadržkov pojasnil situacijo in raz...,N,NaN,annotator_A,E,NaN,annotator_G,N,NaN,annotator_C,N,neutral
4388,"Kot je povedal, naj bi novi načrt podpisal tak...",Sam se na vprašanje o podpisu novih načrtov ni...,C,NaN,annotator_A,C,NaN,annotator_C,NaN,NaN,NaN,C,contradiction
4389,Večina delničarjev ga je potrdila za predsedni...,Banka Karantanija je po izboru večine delničar...,E,NaN,annotator_E,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment
4390,Večina delničarjev ga je potrdila za predsedni...,Trdo delo Janeza Pestotnika se mu je končno ob...,N,NaN,annotator_E,N,NaN,annotator_A,NaN,NaN,NaN,N,neutral


In [11]:
# 5. Convert all columns to uppercase.
df.columns = [i.upper() for i in df.columns]
df.head()

,PAIR_ID,PREMISE,HYPOTHESIS,ANNOTATION_1,COMMENT_1,ANNOTATOR1_ID,ANNOTATION_2,COMMENT_2,ANNOTATOR2_ID,ANNOTATION_3,COMMENT_3,ANNOTATOR3_ID,ANNOTATION_FINAL,LABEL
0,P0,Vendar se je anglikanska večina v grofijah na ...,A na glasovanju o priključitvi ozemlja k Sever...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment
1,P1,INŠTRUKTOR IZ PRTLJAŽNIKA V DRUGO POTOVALKO PR...,Učitelj je vzel iz prtljažnika iz prve potoval...,C,NaN,annotator_B,N,NaN,annotator_A,NaN,NaN,NaN,NaN,contradiction
2,P2,biotska raznovrstnost – v splošnem je to razno...,Četudi je biodiverziteta pomemben del biološke...,C,NaN,annotator_G,N,Morda neustrezen primer? (zdi se kot alineja),annotator_A,NaN,NaN,NaN,NaN,contradiction
3,P3,"Preroški pomen: Če v sanjah bedite, je to na s...",V preroškem smislu budnost v sanjah nakazuje o...,E,Pomanjkanje konteksta?,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment
4,P4,"Jeseni so dnevi krajši, stemni se že dokaj zgo...",V krajših jesenskih dneh tema nastopi relativn...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment


In [12]:
# 6. Replace ```_``` with ```-``` in column names.
df.columns = [i.replace('_', '-') for i in df.columns]
df.head()

,PAIR-ID,PREMISE,HYPOTHESIS,ANNOTATION-1,COMMENT-1,ANNOTATOR1-ID,ANNOTATION-2,COMMENT-2,ANNOTATOR2-ID,ANNOTATION-3,COMMENT-3,ANNOTATOR3-ID,ANNOTATION-FINAL,LABEL
0,P0,Vendar se je anglikanska večina v grofijah na ...,A na glasovanju o priključitvi ozemlja k Sever...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment
1,P1,INŠTRUKTOR IZ PRTLJAŽNIKA V DRUGO POTOVALKO PR...,Učitelj je vzel iz prtljažnika iz prve potoval...,C,NaN,annotator_B,N,NaN,annotator_A,NaN,NaN,NaN,NaN,contradiction
2,P2,biotska raznovrstnost – v splošnem je to razno...,Četudi je biodiverziteta pomemben del biološke...,C,NaN,annotator_G,N,Morda neustrezen primer? (zdi se kot alineja),annotator_A,NaN,NaN,NaN,NaN,contradiction
3,P3,"Preroški pomen: Če v sanjah bedite, je to na s...",V preroškem smislu budnost v sanjah nakazuje o...,E,Pomanjkanje konteksta?,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment
4,P4,"Jeseni so dnevi krajši, stemni se že dokaj zgo...",V krajših jesenskih dneh tema nastopi relativn...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment


In [13]:
# 7. Select rows that belong to the ```neutral``` label.
df = pd.read_csv('SI-NLI/train.tsv', sep='\t')  # reload
df_neutral = df[df['label'] == 'neutral']
df_neutral

,pair_id,premise,hypothesis,annotation_1,comment_1,annotator1_id,annotation_2,comment_2,annotator2_id,annotation_3,comment_3,annotator3_id,annotation_FINAL,label
6,P6,Dopoldne se je na Obrežje s tovornim vozilom a...,Zgodaj dopoldne se je na mejni prehod Obrežje ...,N,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,NaN,neutral
7,P7,Med vsemi pravilnimi odgovori smo izžrebali pe...,Med kar 106 pravilnimi odgovori bo s paketkom ...,N,NaN,annotator_D,N,NaN,annotator_A,NaN,NaN,NaN,N,neutral
9,P9,Bara srednjo dečelco: »Kaku pa ti pojaš?«,Srednji izmed treh punčk je bilo postavljeno v...,N,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,NaN,neutral
13,P13,"DRVA, mešana, kratko žagana z dostavo, prodamo.","Prodajamo različne vrste drv, ki jih lahko nar...",N,NaN,annotator_B,N,NaN,annotator_A,NaN,NaN,NaN,N,neutral
14,P14,"Vso prvo polovico meseca boste čutili, da ima ...",V prvi polovici prvega tedna v mesecu boste na...,N,NaN,annotator_C,C,NaN,annotator_A,NaN,NaN,NaN,NaN,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4377,P4377,"Vsi napovedani dogodki so brezplačni, potrebno...",Kmalu si bo na vrtu možno ogledati vrsto cenov...,N,NaN,annotator_B,E,NaN,annotator_A,N,NaN,annotator_C,N,neutral
4382,P4382,Pravica do odmere dohodnine in drugih davkov p...,"Ko preteče pet let po letu, ko naj bi bili upr...",N,NaN,annotator_D,N,NaN,annotator_C,NaN,NaN,NaN,N,neutral
4383,P4383,opis: Dva “kazuca“ (kozolca); na njih je obeše...,Tradicionalni način sušenja pšenice: postavijo...,N,NaN,annotator_D,N,NaN,annotator_B,NaN,NaN,NaN,N,neutral
4387,P4387,"Kot je povedal, naj bi novi načrt podpisal tak...",Sam je brez zadržkov pojasnil situacijo in raz...,N,NaN,annotator_A,E,NaN,annotator_G,N,NaN,annotator_C,N,neutral


In [14]:
# 8. Select last 30 rows.
df.tail(30)

,pair_id,premise,hypothesis,annotation_1,comment_1,annotator1_id,annotation_2,comment_2,annotator2_id,annotation_3,comment_3,annotator3_id,annotation_FINAL,label
4362,P4362,"Spreminjajo se v času in prostoru, v katerem p...",Največ sprememb pri posamezniku opazimo v njeg...,N,NaN,annotator_D,N,NaN,annotator_C,NaN,NaN,NaN,N,neutral
4363,P4363,"Z ukazom View, Design View jo vrnemo v prikaz ...","V kolikor bomo izbrali ukaz Design View, bomo ...",C,NaN,annotator_A,C,NaN,annotator_B,NaN,NaN,NaN,C,contradiction
4364,P4364,"Z ukazom View, Design View jo vrnemo v prikaz ...",Z desnim klikom na okence View bomo lahko izbr...,N,NaN,annotator_A,N,NaN,annotator_B,NaN,NaN,NaN,N,neutral
4365,P4365,"Z ukazom View, Design View jo vrnemo v prikaz ...","Že želimo pred seboj imeti prikaz Design, poiš...",E,NaN,annotator_A,E,NaN,annotator_G,NaN,NaN,NaN,E,entailment
4366,P4366,Tura je težavna predvsem zaradi dolžine in obi...,"Za pot je potrebno imeti dobro kondicijo, saj ...",N,NaN,annotator_B,E,NaN,annotator_A,N,NaN,annotator_C,N,neutral
4367,P4367,Tura je težavna predvsem zaradi dolžine in obi...,"Tura bo enostavna, saj je precej kratka, hkrat...",C,NaN,annotator_B,C,NaN,annotator_A,NaN,NaN,NaN,C,contradiction
4368,P4368,Tura je težavna predvsem zaradi dolžine in obi...,Pot je težka v veliki meri zaradi svoje dolžin...,E,NaN,annotator_B,E,NaN,annotator_C,NaN,NaN,NaN,E,entailment
4369,P4369,"Evro na trimesečnem vrhu, nafta nad 82 dolarji...","Zadnje časa gre evru zelo slabo, saj je drasti...",C,NaN,annotator_G,C,NaN,annotator_B,NaN,NaN,NaN,C,contradiction
4370,P4370,"Evro na trimesečnem vrhu, nafta nad 82 dolarji...",Evro je v zadnjih treh mesecih pristal na vrhu...,E,NaN,annotator_G,E,NaN,annotator_D,NaN,NaN,NaN,E,entailment
4371,P4371,"Ti laski so edini spomin, ki mi je ostal nate.","Ko si za vselej odšel, sem si shranil tele las...",N,NaN,annotator_D,N,NaN,annotator_A,NaN,NaN,NaN,N,neutral


In [15]:
# 9. Select rows with ```hypothesis``` longer than 100 characters.
long_hypo_mask = df['hypothesis'].apply(lambda s: len(s) > 100)
long_hypo = df[long_hypo_mask]

# check
print(long_hypo['hypothesis'].apply(len))

0       161
1       180
2       143
6       118
8       110
       ... 
4387    122
4388    113
4389    121
4390    145
4391    123
Name: hypothesis, Length: 1720, dtype: int64


In [16]:
# 10. Select rows with ```hypothesis``` longer than 100 characters and belong to the ```neutral``` label.
long_hypo_mask = df['hypothesis'].apply(lambda s: len(s) > 100)
neutral_label_mask = df['label'] == 'neutral'
final_df = df[long_hypo_mask & neutral_label_mask]
final_df

,pair_id,premise,hypothesis,annotation_1,comment_1,annotator1_id,annotation_2,comment_2,annotator2_id,annotation_3,comment_3,annotator3_id,annotation_FINAL,label
6,P6,Dopoldne se je na Obrežje s tovornim vozilom a...,Zgodaj dopoldne se je na mejni prehod Obrežje ...,N,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,NaN,neutral
14,P14,"Vso prvo polovico meseca boste čutili, da ima ...",V prvi polovici prvega tedna v mesecu boste na...,N,NaN,annotator_C,C,NaN,annotator_A,NaN,NaN,NaN,NaN,neutral
24,P24,Madžari petih minut prednosti niso izkoristili...,Madžari pa so na jeseniški turnir poslali poml...,N,NaN,annotator_G,N,"Hipoteza vključuje nove informacije, ki jih iz...",annotator_A,NaN,NaN,NaN,N,neutral
33,P33,(2) Začasno hranjeno carinsko blago se lahko h...,(2) Carinsko blago se lahko začasno hrani do d...,N,NaN,annotator_D,N,NaN,annotator_A,NaN,NaN,NaN,N,neutral
36,P36,"Davčni organ mora o tem izdati odločbo, s kate...","Zakon določa, da se v primeru, če dolžnikov do...",N,NaN,annotator_G,N,NaN,annotator_A,NaN,NaN,NaN,N,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,P4356,"Pri čelnih trkih se je izkazalo, da so pri trk...",Če pogledamo statistiko prometnih nesreč v zad...,N,NaN,annotator_A,N,NaN,annotator_C,NaN,NaN,NaN,N,neutral
4360,P4360,"Spreminjajo se v času in prostoru, v katerem p...","Ljudje se najbolj spreminjajo, če se spremeni ...",E,NaN,annotator_D,N,NaN,annotator_A,N,NaN,annotator_C,N,neutral
4382,P4382,Pravica do odmere dohodnine in drugih davkov p...,"Ko preteče pet let po letu, ko naj bi bili upr...",N,NaN,annotator_D,N,NaN,annotator_C,NaN,NaN,NaN,N,neutral
4387,P4387,"Kot je povedal, naj bi novi načrt podpisal tak...",Sam je brez zadržkov pojasnil situacijo in raz...,N,NaN,annotator_A,E,NaN,annotator_G,N,NaN,annotator_C,N,neutral


In [17]:
# 11. Select the row with the longest ```hypothesis```.
df['hypo_len'] = df['hypothesis'].apply(len)
df[df['hypo_len'] == df['hypo_len'].max()]

,pair_id,premise,hypothesis,annotation_1,comment_1,annotator1_id,annotation_2,comment_2,annotator2_id,annotation_3,comment_3,annotator3_id,annotation_FINAL,label,hypo_len
3847,P3847,DOBRO JUTRO Z RADIO CAPODISTRIA 07.40 Literarn...,JUTRANJA ODDAJA KOPRSKEGA RADIA 20 do osmih zj...,E,NaN,annotator_D,E,NaN,annotator_I,NaN,NaN,NaN,E,entailment,326


In [18]:
# 12. Remove rows that contain ```č```, ```š```, ```ž``` in ```premise``` or ```hypothesis```.
def check(s):
    for c in s.lower():
        if c in chars:
            return False
    return True

chars = ['č', 'š', 'ž']
premise_mask = df['premise'].apply(check)
hypo_mask = df['hypothesis'].apply(check)
df[premise_mask & hypo_mask]

,pair_id,premise,hypothesis,annotation_1,comment_1,annotator1_id,annotation_2,comment_2,annotator2_id,annotation_3,comment_3,annotator3_id,annotation_FINAL,label,hypo_len
62,P62,"Tisti, ki se imajo radi, se ne bojijo, da jih ...","Kdor ima rad sebe, je zaskrbljen, kako ga vidi...",C,NaN,annotator_C,C,NaN,annotator_A,NaN,NaN,NaN,C,contradiction,56
63,P63,"Oblival ga je hladen pot, vendar je naposled u...",Ne glede na vse napore ni uspel.,C,NaN,annotator_C,C,NaN,annotator_A,NaN,NaN,NaN,C,contradiction,32
69,P69,»Da nas ne bodo ujeli na nadzorne kamere.«,"“Upam, da nas ne bodo posneli na njihove varno...",E,NaN,annotator_G,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,59
115,P115,"»Zelo ste prijazni,« se je nasmehnila Kyla.","“Izjemno nesramni ste,“ je negodovala Kyla.",C,NaN,annotator_C,C,NaN,annotator_A,NaN,NaN,NaN,C,contradiction,43
126,P126,"Pogled se ji je stisnil, obraz se ji je stisnil.","Ko sva govorila sem videl, da ima nekaj v mislih.",N,NaN,annotator_G,N,NaN,annotator_A,NaN,NaN,NaN,N,neutral,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4290,P4290,Popoldne so med poostrenim nadzorom prometa v ...,Voznik je bil alkoholik in stari znanec policije.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,neutral,49
4291,P4291,Popoldne so med poostrenim nadzorom prometa v ...,"Med nadziranjem prometa so ustavili Dolenjca, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,neutral,118
4327,P4327,"Mami mi je obljubila, da mi bo naredila pramene.«","“Mami mi je dala besedo, da mi bo napravila pr...",E,NaN,annotator_B,E,NaN,annotator_C,NaN,NaN,NaN,E,entailment,53
4363,P4363,"Z ukazom View, Design View jo vrnemo v prikaz ...","V kolikor bomo izbrali ukaz Design View, bomo ...",C,NaN,annotator_A,C,NaN,annotator_B,NaN,NaN,NaN,C,contradiction,70


In [19]:
# 13. Remove rows that contain at least one missing value.
df.dropna(axis=0, how='any')  # all rows contain at least one missing value
df.dropna(axis=1, how='any')  # this is not true for columns

,pair_id,premise,hypothesis,label,hypo_len
0,P0,Vendar se je anglikanska večina v grofijah na ...,A na glasovanju o priključitvi ozemlja k Sever...,entailment,161
1,P1,INŠTRUKTOR IZ PRTLJAŽNIKA V DRUGO POTOVALKO PR...,Učitelj je vzel iz prtljažnika iz prve potoval...,contradiction,180
2,P2,biotska raznovrstnost – v splošnem je to razno...,Četudi je biodiverziteta pomemben del biološke...,contradiction,143
3,P3,"Preroški pomen: Če v sanjah bedite, je to na s...",V preroškem smislu budnost v sanjah nakazuje o...,entailment,57
4,P4,"Jeseni so dnevi krajši, stemni se že dokaj zgo...",V krajših jesenskih dneh tema nastopi relativn...,entailment,55
...,...,...,...,...,...
4387,P4387,"Kot je povedal, naj bi novi načrt podpisal tak...",Sam je brez zadržkov pojasnil situacijo in raz...,neutral,122
4388,P4388,"Kot je povedal, naj bi novi načrt podpisal tak...",Sam se na vprašanje o podpisu novih načrtov ni...,contradiction,113
4389,P4389,Večina delničarjev ga je potrdila za predsedni...,Banka Karantanija je po izboru večine delničar...,entailment,121
4390,P4390,Večina delničarjev ga je potrdila za predsedni...,Trdo delo Janeza Pestotnika se mu je končno ob...,neutral,145


In [20]:
# 14. Remove the column with the most missing values.
col_position = df.isna().sum().argmax()
df.drop(columns=[df.columns[col_position]])

,pair_id,premise,hypothesis,annotation_1,comment_1,annotator1_id,annotation_2,comment_2,annotator2_id,annotation_3,annotator3_id,annotation_FINAL,label,hypo_len
0,P0,Vendar se je anglikanska večina v grofijah na ...,A na glasovanju o priključitvi ozemlja k Sever...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,E,entailment,161
1,P1,INŠTRUKTOR IZ PRTLJAŽNIKA V DRUGO POTOVALKO PR...,Učitelj je vzel iz prtljažnika iz prve potoval...,C,NaN,annotator_B,N,NaN,annotator_A,NaN,NaN,NaN,contradiction,180
2,P2,biotska raznovrstnost – v splošnem je to razno...,Četudi je biodiverziteta pomemben del biološke...,C,NaN,annotator_G,N,Morda neustrezen primer? (zdi se kot alineja),annotator_A,NaN,NaN,NaN,contradiction,143
3,P3,"Preroški pomen: Če v sanjah bedite, je to na s...",V preroškem smislu budnost v sanjah nakazuje o...,E,Pomanjkanje konteksta?,annotator_C,E,NaN,annotator_A,NaN,NaN,E,entailment,57
4,P4,"Jeseni so dnevi krajši, stemni se že dokaj zgo...",V krajših jesenskih dneh tema nastopi relativn...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,E,entailment,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4387,P4387,"Kot je povedal, naj bi novi načrt podpisal tak...",Sam je brez zadržkov pojasnil situacijo in raz...,N,NaN,annotator_A,E,NaN,annotator_G,N,annotator_C,N,neutral,122
4388,P4388,"Kot je povedal, naj bi novi načrt podpisal tak...",Sam se na vprašanje o podpisu novih načrtov ni...,C,NaN,annotator_A,C,NaN,annotator_C,NaN,NaN,C,contradiction,113
4389,P4389,Večina delničarjev ga je potrdila za predsedni...,Banka Karantanija je po izboru večine delničar...,E,NaN,annotator_E,E,NaN,annotator_A,NaN,NaN,E,entailment,121
4390,P4390,Večina delničarjev ga je potrdila za predsedni...,Trdo delo Janeza Pestotnika se mu je končno ob...,N,NaN,annotator_E,N,NaN,annotator_A,NaN,NaN,N,neutral,145


## Create new variables


1. Create integer type variable ```vowel_count_premise``` which stores the number of vowels in a ```premise```. Repeat for ```hypothesis```.
2. Create integer type variable with possible values ```1```, ```2```, ```3``` that counts how many annotations a single example received.
3. Create boolean type variable ```agreement``` which reflects whether all annotators agreed on the label.

In [21]:
# 1. Create integer type variable ```vowel_count_premise``` which stores the number of vowels in a ```premise```. Repeat for ```hypothesis```.
def count_vowels(s):
    n = 0
    for c in s.lower():
        if c in vowels:
            n += 1
    return n

vowels = {'a', 'e', 'i', 'o', 'u'}
df['premise_vowels'] = df['premise'].apply(count_vowels)
df['hypothesis_vowels'] = df['hypothesis'].apply(count_vowels)
df.head()

,pair_id,premise,hypothesis,annotation_1,comment_1,annotator1_id,annotation_2,comment_2,annotator2_id,annotation_3,comment_3,annotator3_id,annotation_FINAL,label,hypo_len,premise_vowels,hypothesis_vowels
0,P0,Vendar se je anglikanska večina v grofijah na ...,A na glasovanju o priključitvi ozemlja k Sever...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,161,46,54
1,P1,INŠTRUKTOR IZ PRTLJAŽNIKA V DRUGO POTOVALKO PR...,Učitelj je vzel iz prtljažnika iz prve potoval...,C,NaN,annotator_B,N,NaN,annotator_A,NaN,NaN,NaN,NaN,contradiction,180,50,56
2,P2,biotska raznovrstnost – v splošnem je to razno...,Četudi je biodiverziteta pomemben del biološke...,C,NaN,annotator_G,N,Morda neustrezen primer? (zdi se kot alineja),annotator_A,NaN,NaN,NaN,NaN,contradiction,143,43,51
3,P3,"Preroški pomen: Če v sanjah bedite, je to na s...",V preroškem smislu budnost v sanjah nakazuje o...,E,Pomanjkanje konteksta?,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,57,24,17
4,P4,"Jeseni so dnevi krajši, stemni se že dokaj zgo...",V krajših jesenskih dneh tema nastopi relativn...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,55,16,17


In [22]:
# 2. Create integer type variable with possible values ```1```, ```2```, ```3``` that counts how many annotations a single example received.
df['num_of_annotations'] = df[['annotator1_id', 'annotator2_id', 'annotator3_id']].notna().sum(axis=1)
df.head()

,pair_id,premise,hypothesis,annotation_1,comment_1,annotator1_id,annotation_2,comment_2,annotator2_id,annotation_3,comment_3,annotator3_id,annotation_FINAL,label,hypo_len,premise_vowels,hypothesis_vowels,num_of_annotations
0,P0,Vendar se je anglikanska večina v grofijah na ...,A na glasovanju o priključitvi ozemlja k Sever...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,161,46,54,2
1,P1,INŠTRUKTOR IZ PRTLJAŽNIKA V DRUGO POTOVALKO PR...,Učitelj je vzel iz prtljažnika iz prve potoval...,C,NaN,annotator_B,N,NaN,annotator_A,NaN,NaN,NaN,NaN,contradiction,180,50,56,2
2,P2,biotska raznovrstnost – v splošnem je to razno...,Četudi je biodiverziteta pomemben del biološke...,C,NaN,annotator_G,N,Morda neustrezen primer? (zdi se kot alineja),annotator_A,NaN,NaN,NaN,NaN,contradiction,143,43,51,2
3,P3,"Preroški pomen: Če v sanjah bedite, je to na s...",V preroškem smislu budnost v sanjah nakazuje o...,E,Pomanjkanje konteksta?,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,57,24,17,2
4,P4,"Jeseni so dnevi krajši, stemni se že dokaj zgo...",V krajših jesenskih dneh tema nastopi relativn...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,55,16,17,2


In [23]:
# 3. Create boolean type variable ```agreement``` which reflects whether all annotators agreed on the label.
values = []
for idx, row in df[['annotation_1', 'annotation_2', 'annotation_3']].iterrows():
    row = row.dropna()  # drop na from a row
    s = set(row.to_dict().values())
    if len(s) == 1:
        values.append(True)
    else:
        values.append(False)
df['agree'] = values
df

,pair_id,premise,hypothesis,annotation_1,comment_1,annotator1_id,annotation_2,comment_2,annotator2_id,annotation_3,comment_3,annotator3_id,annotation_FINAL,label,hypo_len,premise_vowels,hypothesis_vowels,num_of_annotations,agree
0,P0,Vendar se je anglikanska večina v grofijah na ...,A na glasovanju o priključitvi ozemlja k Sever...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,161,46,54,2,True
1,P1,INŠTRUKTOR IZ PRTLJAŽNIKA V DRUGO POTOVALKO PR...,Učitelj je vzel iz prtljažnika iz prve potoval...,C,NaN,annotator_B,N,NaN,annotator_A,NaN,NaN,NaN,NaN,contradiction,180,50,56,2,False
2,P2,biotska raznovrstnost – v splošnem je to razno...,Četudi je biodiverziteta pomemben del biološke...,C,NaN,annotator_G,N,Morda neustrezen primer? (zdi se kot alineja),annotator_A,NaN,NaN,NaN,NaN,contradiction,143,43,51,2,False
3,P3,"Preroški pomen: Če v sanjah bedite, je to na s...",V preroškem smislu budnost v sanjah nakazuje o...,E,Pomanjkanje konteksta?,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,57,24,17,2,True
4,P4,"Jeseni so dnevi krajši, stemni se že dokaj zgo...",V krajših jesenskih dneh tema nastopi relativn...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,55,16,17,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4387,P4387,"Kot je povedal, naj bi novi načrt podpisal tak...",Sam je brez zadržkov pojasnil situacijo in raz...,N,NaN,annotator_A,E,NaN,annotator_G,N,NaN,annotator_C,N,neutral,122,34,39,3,False
4388,P4388,"Kot je povedal, naj bi novi načrt podpisal tak...",Sam se na vprašanje o podpisu novih načrtov ni...,C,NaN,annotator_A,C,NaN,annotator_C,NaN,NaN,NaN,C,contradiction,113,34,37,2,True
4389,P4389,Večina delničarjev ga je potrdila za predsedni...,Banka Karantanija je po izboru večine delničar...,E,NaN,annotator_E,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,121,44,45,2,True
4390,P4390,Večina delničarjev ga je potrdila za predsedni...,Trdo delo Janeza Pestotnika se mu je končno ob...,N,NaN,annotator_E,N,NaN,annotator_A,NaN,NaN,NaN,N,neutral,145,44,50,2,True


## Combine datasets

1. Import dev and test files.
2. Combine all three splits into one large dataset.
3. What is the average length of ```premise``` per label?
4. How many examples each split contains?
5. Create a subset that contains exactly the same number of examples per split.

In [24]:
# 1. Import dev and test files.
train = pd.read_csv('SI-NLI/train.tsv', sep='\t')
train['split'] = ['train']*len(train)
dev = pd.read_csv('SI-NLI/dev.tsv', sep='\t')
dev['split'] = ['dev']*len(dev)
test = pd.read_csv('SI-NLI/test.tsv', sep='\t')
test['split'] = ['test']*len(test)

In [25]:
# 2. Combine all three splits into one large dataset.
df = pd.concat([train, dev, test])
df

,pair_id,premise,hypothesis,annotation_1,comment_1,annotator1_id,annotation_2,comment_2,annotator2_id,annotation_3,comment_3,annotator3_id,annotation_FINAL,label,split
0,P0,Vendar se je anglikanska večina v grofijah na ...,A na glasovanju o priključitvi ozemlja k Sever...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,train
1,P1,INŠTRUKTOR IZ PRTLJAŽNIKA V DRUGO POTOVALKO PR...,Učitelj je vzel iz prtljažnika iz prve potoval...,C,NaN,annotator_B,N,NaN,annotator_A,NaN,NaN,NaN,NaN,contradiction,train
2,P2,biotska raznovrstnost – v splošnem je to razno...,Četudi je biodiverziteta pomemben del biološke...,C,NaN,annotator_G,N,Morda neustrezen primer? (zdi se kot alineja),annotator_A,NaN,NaN,NaN,NaN,contradiction,train
3,P3,"Preroški pomen: Če v sanjah bedite, je to na s...",V preroškem smislu budnost v sanjah nakazuje o...,E,Pomanjkanje konteksta?,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,train
4,P4,"Jeseni so dnevi krajši, stemni se že dokaj zgo...",V krajših jesenskih dneh tema nastopi relativn...,E,NaN,annotator_C,E,NaN,annotator_A,NaN,NaN,NaN,E,entailment,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,NaN,"»Spakirali bomo vse stvari, ki jih moramo vzet...","“Za vso prtljago, ki jo potrebujemo, smo že po...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
994,NaN,"»Spakirali bomo vse stvari, ki jih moramo vzet...","“Pripraviti si moramo vse, kar bomo potrebovali.“",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
995,NaN,S 1. januarjem 2008 je selektorsko mesto prevz...,Pri selektorskem mestu v letu 2008 ni prišlo d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
996,NaN,S 1. januarjem 2008 je selektorsko mesto prevz...,"Od leta 2008, ko je selektorsko mesto zasedla ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test


In [26]:
# 3. What is the average length of ```premise``` per label?
df['premise_length'] = df['premise'].apply(len)
group = df.groupby(by='label')
group['premise_length'].mean()

label
contradiction    92.984039
entailment       93.521333
neutral          92.954972
Name: premise_length, dtype: float64

In [27]:
# 4. How many examples each split contains?
df['label'].value_counts()

label
entailment       1711
contradiction    1629
neutral          1599
Name: count, dtype: int64

In [28]:
# 5. Create a subset that contains exactly the same number of examples per split.
train_s = train.sample(n=100)
dev_s = dev.sample(n=100)
test_s = test.sample(n=100)

subset = pd.concat([train_s, dev_s, test_s])
subset['split'].value_counts()

split
train    100
dev      100
test     100
Name: count, dtype: int64

## Save dataframes

1. Save the original dataset to disk in a ```csv``` format.

In [29]:
# 1. Save the original dataset to disk in a ```csv``` format.
df = pd.read_csv('SI-NLI/train.tsv', sep='\t')
df.to_csv('SI-NLI/train.csv', index=False)